# Sentinel Hub download data

## 0. Install and import packages

In [ ]:
!pip install suntime
!pip install creds
!pip install sentinelhub

In [ ]:
import os, shutil
import getpass
import requests
import pandas as pd
from datetime import datetime as dt
from suntime import Sun
from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    SentinelHubCatalog,
    )

## 1. Setting up credentials

### 1.1 Obtain token

Sign up in https://dataspace.copernicus.eu/

In [ ]:
copernicus_user = str(input('Copernicus user: '))
copernicus_password = getpass.getpass("Password: ")

def get_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
        )
    return r.json()["access_token"]

### 1.2 Catalog config

A SentinelHub client must be configured (https://www.youtube.com/watch?v=GA50fqbrCo4&list=PLj4KwRQTBlPL3CVwpaBXl3VrO0_V3VX_Q&pp=iAQB)

In [ ]:
config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")
catalog = SentinelHubCatalog(config=config)

## 2. Obtain data

*   **aoi_coords_wgs84:** BBox of the area of interest with shape (lon1, lat1, lon2, lat2). Can easily be obtained in http://bboxfinder.com/
*   **time_interval:** Defines the start and end dates to download data. date = "YYYY-mm-dd"; time_interval = start_date, end_date 

In [ ]:
# Define area of interest and time interval

aoi_coords_wgs84 = (71.5034, 70.2289, 71.9717, 70.2820) # (lon1, lat1, lon2, lat2)
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
time_interval = "2017-06-20", "2018-01-15"

# Search filtered data

search_iterator = catalog.search(
    DataCollection.SENTINEL3_SLSTR,
    bbox=aoi_bbox,
    time=time_interval,
    filter="eo:cloud_cover < 35",
    fields={"include": ["id", "properties.datetime"], "exclude": []},
    )

results = list(search_iterator)
print("Total number of results:", len(results))


# Day time observations

date_start = dt.strptime(time_interval[0], '%Y-%m-%d')
sun = Sun(aoi_coords_wgs84[1], aoi_coords_wgs84[0]) # Latitude and longitude of observation
sunrise_utc = sun.get_sunrise_time(date_start).time()
sunset_utc = sun.get_sunset_time(date_start).time()

all_uuids_to_download = []

for key in results:
    date = key['properties']['datetime'].replace('T', ' ').split('.')[0]
    date = dt.strptime(date, '%Y-%m-%d %H:%M:%S')
    time = date.time()
    if sunset_utc > sunrise_utc:
        if time > sunrise_utc and time < sunset_utc:
            all_uuids_to_download.append(key['id'])
    else:
        if time > sunrise_utc or time < sunset_utc:
            all_uuids_to_download.append(key['id'])

print("Total day time observations:", len(all_uuids_to_download))

In [ ]:
# Obtain the Id of the requested observations

data_collection = "SENTINEL-3"
Id_to_download = []

for uuid in all_uuids_to_download:
  json = requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and contains(Name,'{uuid}')").json()
  pd.DataFrame.from_dict(json['value']).head(4)
  df = pd.DataFrame.from_dict(json['value'])
  Id_to_download.append(df['Id'][0])

## 3. Download data
*   **download_dir:** directory where data will be downloaded

In [ ]:
# Download directory

download_dir = './Data/Case_name/'

# Download process

token = get_token(copernicus_user, copernicus_password)

print(f'Downloading {len(Id_to_download)} files:')

for i in range(len(Id_to_download)):
  url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({Id_to_download[i]})/$value"
  print(f'  {i+1}. Downloading {all_uuids_to_download[i]}.zip')

  headers = {"Authorization": f"Bearer {token}"}

  session = requests.Session()
  session.headers.update(headers)
  response = session.get(url, headers=headers, stream=True)
  with open(f"{download_dir}{all_uuids_to_download[i]}.zip", "wb") as file:
      for chunk in response.iter_content(chunk_size=8192):
          if chunk:
              file.write(chunk)

# Unzip the product files

print('Start unzipping ... ')

for filename in os.listdir(download_dir):
    if filename.endswith('.zip'):
        shutil.unpack_archive(download_dir + filename, download_dir)
        os.remove(download_dir + filename)

print(' ... Finished unzipping')